SETUP


In [1]:
#import all necessary libraries
import azure.cognitiveservices.speech as speechsdk
import time

In [2]:
#initialize speech config
speech_config = speechsdk.SpeechConfig(
        subscription='44d7d58744334bea8b7c72de640ed3e3',
        region='southeastasia')
speech_config.speech_recognition_language = "en-US"

FROM MIC #1

In [4]:
def recognize_from_microphone():
    audio_config = speechsdk.audio.AudioConfig(use_default_microphone=True)
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config,
                                                   audio_config=audio_config)

    print("Speak into your microphone.")
    speech_recognition_result = speech_recognizer.recognize_once_async().get()

    if speech_recognition_result.reason == speechsdk.ResultReason.RecognizedSpeech:
        print("Recognized: {}".format(speech_recognition_result.text))
    elif speech_recognition_result.reason == speechsdk.ResultReason.NoMatch:
        print("No speech could be recognized: {}".format(
            speech_recognition_result.no_match_details))
    elif speech_recognition_result.reason == speechsdk.ResultReason.Canceled:
        cancellation_details = speech_recognition_result.cancellation_details
        print("Speech Recognition canceled: {}".format(
            cancellation_details.reason))
        if cancellation_details.reason == speechsdk.CancellationReason.Error:
            print("Error details: {}".format(
                cancellation_details.error_details))
            print("Did you set the speech resource key and region values?")

In [5]:
recognize_from_microphone()

Speak into your microphone.
Recognized: Test.


FROM MIC #2

In [3]:
def from_mic():
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config)

    print("Speak into your microphone.")
    result = speech_recognizer.recognize_once_async().get()
    if result.reason == speechsdk.ResultReason.RecognizedSpeech:
        print("Recognized: {}".format(result.text))
    elif result.reason == speechsdk.ResultReason.NoMatch:
        print("No speech could be recognized: {}".format(result.no_match_details))
    elif result.reason == speechsdk.ResultReason.Canceled:
        cancellation_details = result.cancellation_details
        print("Speech Recognition canceled: {}".format(cancellation_details.reason))
        if cancellation_details.reason == speechsdk.CancellationReason.Error:
            print("Error details: {}".format(cancellation_details.error_details))
            print("Did you set the speech resource key and region values?")

In [6]:
from_mic()

Speak into your microphone.
Recognized: Can you run hello?


Continuous Speech Recognition????


In [ ]:
def continuous_recog():
    audio_config = speechsdk.audio.AudioConfig(use_default_microphone=True)
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)
    done = False

    def stop_cb(evt):
        print('CLOSING on {}'.format(evt))
        speech_recognizer.stop_continuous_recognition()
        nonlocal done
        done = True

    speech_recognizer.recognizing.connect(lambda evt: print('RECOGNIZING: {}'.format(evt)))
    speech_recognizer.recognized.connect(lambda evt: print('RECOGNIZED: {}'.format(evt)))
    speech_recognizer.session_started.connect(lambda evt: print('SESSION STARTED: {}'.format(evt)))
    speech_recognizer.session_stopped.connect(lambda evt: print('SESSION STOPPED {}'.format(evt)))
    speech_recognizer.canceled.connect(lambda evt: print('CANCELED {}'.format(evt)))

    speech_recognizer.session_stopped.connect(stop_cb)
    speech_recognizer.canceled.connect(stop_cb)

    speech_recognizer.start_continuous_recognition()
    while not done:
        time.sleep(.5)
    

Wakeword activation

In [6]:
def wakeWordActivation():
    #if wakeword is not string or empty, return
    model = speechsdk.KeywordRecognitionModel("e907bd72-6dff-4316-bc33-1140bffc4686.table")
    # keyword = 'hey dude whats up'
    audio_config = speechsdk.audio.AudioConfig(use_default_microphone=True)
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config,audio_config=audio_config)
    done = False

    def exit_callback(evt):
        if evt.result.reason == speechsdk.ResultReason.RecognizedKeyword:
            print("RECOGNIZED KEYWORD: {}".format(evt.result.text))
        nonlocal done
        done = True
    
    speech_recognizer.recognized.connect(exit_callback)
    speech_recognizer.start_keyword_recognition(model)
    print('---START---\n')

    while not done:
        time.sleep(.5)

    speech_recognizer.stop_keyword_recognition()

    print('Go ahead, say something')
    result = speech_recognizer.recognize_once_async().get()
    if result.reason == speechsdk.ResultReason.RecognizedSpeech:
        print("Result: {}".format(result.text))
    return result.text


In [4]:
wakeWordActivation()

.:START:.

RECOGNIZED KEYWORD: hey dude whats up
Go ahead, say something
Result: No, I'm gonna put it.
